# <center> PP7 - Solutions

## Querying probabilities

In Python, querying probabilities of continuous distributions in Python is pretty much the same as doing so for discrete distributions. All you have to do is to change the line where you initialize the distribution, replacing it by the appropriate one.

PRO TIP! This exercise will walk you through the most common distributions, but you should always consult the <a href="https://docs.scipy.org/doc/scipy/reference/stats.html">documentation</a> before using the new distribution, to make sure you are initializing it correctly. 

<b>About the Normal</b>

When using the Normal distribution, you should pass the mean and the *standard deviation*, not the variance. For example, even though we write $X \sim Normal(1, 4)$ [mean 1, *variance* 4], in Python you must initialize `norm` as

```python
dist = ss.norm(1, 2)      # Mean 0, **standard deviation** 2
```
<font size=1>By the way, if you use Python you don't have to go through the steps of defining $Z = aX + b$ etc like you did in Q3 of the theory part!

<b>About the Chi-squared and Student t</b>

The parameter that governs these two distributions is called the "degrees of freedom".

```python
dist = ss.chi2(df = 3)  # Chi-squared with 3 'degrees of freedom'
dist = ss.t(df = 7)     # Student-t with 7 'degrees of freedom'
```

---

<font color="blue">Importing libraries first!

In [1]:
import scipy.stats as ss
import numpy as np

## Dropping bombs

An airplane is supposed to depart from its origin, fly East on a straight line, and then drop a bomb at a certain target located 20 miles away. However, the accuracy of the bomber is not perfect, so the position of the bomb attack can be represented by the random variable $X \sim Normal(20, 1)$ -- that is, a Normally distributed rv with mean $20$ and *variance* $1$.

<img src="airplane.png" height = 300>

Q1) Using the object `norm`, find out the probability that the bomb lands within 1km from its target, i.e., $P(19 \leq X \leq 21)$.

<font color="blue">
$P(19 \leq X \leq 21) = F_X(21) - F_X(19)$, therefore

In [7]:
dist = ss.norm(20, 1)
prob = dist.cdf(21) - dist.cdf(19)
print(prob)

0.682689492137


Q2) Let $D$ be the a random variable representing the *squared distance* from the target, i.e., $D = (X - 20)^2$. Find out $P(D \leq 2)$.

<b>Hint</b> Recall that if you square a standard Normal random variable, the result is a Chi-squared random variable with one degree of freedom.

<font color="blue">
The key fact is this: 

> The sum of $n$ squared standard Normals is $\chi^2_n$ ("chi-squared with $n$ degrees of freedom")

Or in symbols:

> If $Z_1, Z_2, \cdots, Z_n \sim Normal(0,1)$, then $Z_1^2 + Z_2^2 + \cdots + Z_n^2 \sim \chi_n^2$

Now, $X$ is not standard Normal...

$X \sim Normal(20, 1)$

...but $X - 20$ is!

$X - 20 \sim Normal(0, 1)$

That's because by subtracting $20$ we're shifting the distribution $20$ to the left. So if before it was centered at $20$, now it's centered at zero. As for the variance, there's nothing to do.

So if we square that...

$(X - 20)^2 \sim \chi^2_1$

...we'll be squaring *one* standard Normal, so the resulting distribution is Chi-squared with *one* degree of freedom.

Put these together, and the code is pretty simple.

In [10]:
dist = ss.chi2(df = 1)
prob = dist.cdf(2)
print(prob)

0.84270079295


<font color="blue"><b>A general two-step rule to always find the solution</b>

Suppose you forgot about the relationship between Chi-squared and Normals. Then what?

<br>
Actually, it's not *that* hard to figure out the resulting distribution. Read on.

You want to know

$\begin{align}
P(D \leq 2) 
&= P((X - 20)^2 \leq 2) \qquad \text{Definition of $D$} \\
&= P(-\sqrt{2} \leq X - 20 \leq \sqrt{2}) \qquad \text{Undoing the squaring} \\
&= P(-\sqrt{2} + 20 \leq X \leq \sqrt{2} + 20) \qquad \text{Undoing the subtraction} \\
\end{align}$

But hey, look at that expression: it's in terms of $X$, whose distribution you do know!

$\begin{align}
P(D \leq 2) 
&= P(-\sqrt{2} + 20 \leq X \leq \sqrt{2} + 20)  \\
&= F_X(\sqrt{2} + 20) -  F_X(-\sqrt{2} + 20) \qquad \text{Using the CDF of $X$} \\
\end{align}$

Let's see if we get the same results

In [18]:
dist = ss.norm(20, 1)
prob = dist.cdf(np.sqrt(2) + 20) - dist.cdf(-np.sqrt(2) + 20)
print(prob)

0.84270079295


<font color="blue"> See? The idea is just to keep undoing all the operations until we get to a probability that's expressed in terms of a random variable whose distribution we know. This is very similar to what you did in $Q3$, where you wanted to know something about $X \sim Normal(3, 4)$, but you had to first re-express it in terms of $Z$, whose distribution you did know. So you rewrote

$$X = 2Z + 3$$

And then computed

$\begin{align}
P(X \leq 1) 
&= P(2Z + 3 \leq 1)  \qquad \text{Definition of Z} \\
&= P(2Z \leq 1 - 3)  \qquad \text{Undoing the addition} \\
&= P\left(Z \leq \frac{1 - 3}{2}\right)  \qquad \text{Undoing the multiplication} \\
\end{align}$

Again, it's a two-step procedure: rewrite the RV whose distribution you don't know in terms of RVs whose distribution you do know, and then *undo* all the operations until you reach an expression you can query from your table or programming library.

...Whew!

Q3) Now let's make the problem just slightly more realistic by putting the target on a two-dimensional plane. This time, the target is at coordinate $(20,20)$, and both the $X$ and $Y$ coordinates are Normally distributed: $X, Y \sim Normal(20, 2)$ and they are *independent*. 


<img src="airplane2.png">

Find out the probability that that bomb lands inside a box of side 2km by 2km, centered at the target, like below.


<img src="airplane3.png">

<font color="blue">
There's two events at play here.

+ $A$: $X$ falls between $[19, 21]$

+ $B$: $Y$ falls between $[19, 21]$

In terms of these events, we're looking for the probability of $A$ AND $B$, or $P(A \cap B)$.

Computing these intersection can actually be hard (i.e., hard enough to be outside the scope of this couse), but in this case it isn't, because $X$ and $Y$ are independent, so:

$\begin{align}
P(A \cap B) 
&= P(A) P(B) \qquad\qquad\qquad\qquad\qquad\qquad   \text{By independence} \\
&= P(19 \leq X \leq 21)\cdot P(19 \leq Y \leq 21) \qquad \qquad \text{Definitions of $A$ and $B$}\\
&= [F_X(21) - F_X(19)] \cdot [F_Y(21) - F_Y(12)]\qquad \qquad \text{Definition of CDF} \\
&= [F_X(21) - F_X(19)]^2 \qquad\qquad\qquad \qquad \text{$X$ and $Y$ have same distribution, hence same CDF}
\end{align}$



In [15]:
dist = ss.norm(20, np.sqrt(2))        # Super annoying, I know: ss.norm takes the mean and *std. deviation* (not variance)
probx = dist.cdf(21) - dist.cdf(19)  
prob = probx **2   
print(prob)

0.270920122803


Q4) Find out the probability that that bomb lands inside *circle* of radius 1km, centered at the target, like below.

<b>Hint</b> The distance between the target and the bomb landing is given by $D = (X -20)^2 + (Y - 20)^2$.

<img src="airplane4.png">

<font color="blue">
Q3 was like Q1, and Q4 is like Q2.

The easiest way to find the solution here is to use the "general two-step procedure" I described in the solution to Q2.

<br>
<b>#1</b> Rewrite $X$, $Y$ in terms of random variables whose probability distribution you know

Let use $Z_1, Z_2 \sim Normal(0,1)$. 

$X = \sqrt{2}Z_1 + 20$ and $Y = \sqrt{2}Z_2 + 20$ 

<b>#2</b> Undo all the operations until you reach something whose probability distribution you do know

$\begin{align}
P((X-20)^2 + (Y - 20)^2 \leq 1) 
&= P(2Z_1^2 + 2Z_2^2 \leq 1) \qquad \text{Definition of }X, Y \\
&= P \left( Z_1^2 + Z_2^2 \leq \frac{1}{2} \right) \qquad \text{Definition of }X, Y \\
\end{align}$

The expression on the left-hand side of the inequality is the sum of two squared standard Normals, hence it's chi-squared with two degrees of freedom. 

In [57]:
dist = ss.chi2(df = 2)
prob = dist.cdf(1/2)
print(prob)

0.221199216929
